Les acteurs ont en moyenne quel âge ? données films et acteurs (champs age, birth year et start year)
***age moyen à date du film

1- Appel des deux tables - trie ordre croissant - reset index et drop colonne en trop pour les deux tables

2- Nettoyage de la base Movies :
         - Conservation uniquement des films titleType = movie
         - Retrait des films isadult
         - Suppresion colonne quine me sont plus utiles
         - Explode genre en plusieurs colonnes

3- Nettoyage de la base Actors :
        - Supprimer les acteurs nés avant 1800 pour alléger la base
        - Explode colonne primaryProfession et conserver actor actress
        - Retirer la colonne deathYear
        - Explode colonne knownForTitles en film 1 film 2 etc....
        
        


3- Joindre les tables et retrier :
    - Supprimer les films plus anciens que 1918
    - Suprimer les lignes ou l'âge de l'acteur n'est pas renseigné
(ce tri n'a pas été fait avant car cela permet de nattoyer la base acteurs/movies en même temps)

4 - Créer une colonne âge acteur dans le film : Date du film - date de naissance (startYear - birthYear)

5 - il me faudra une moyenne de cette colonne pour répondre à la question par année






In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

## 1- Appel des trois tables - trie - reset index et drop colonne en trop pour les deux tables :
 actors = name.basics
 movies = title.basics
 title = 

## Table Actors

In [2]:
# appel de la table acteurs

actors = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', low_memory=False)

In [3]:
# tri des acteurs par date de naissance 
actors.sort_values(by=['birthYear'], inplace=True)

#reset index 
actors.reset_index(inplace=True)

#drop colonne index en double 
actors.drop('index',axis = 1, inplace=True)



In [4]:
# check si l'ordre est bien modifié et l'index modifié

actors.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0784172,Lucio Anneo Seneca,0004,0065,writer,"tt0218822,tt0049203,tt0237666,tt0972562"
1,nm4511421,Megan Liz Smith,0012,\N,"miscellaneous,actress","tt1966378,tt10183848,tt3416744"
2,nm8810146,Paul Walsh,0021,\N,actor,"tt0314567,tt0093223,tt0098878"
3,nm0430769,Flavius Josephus,0037,0095,"writer,miscellaneous","tt0009573,tt0237666,tt0049833,tt0273236"
4,nm1306202,Plutarch,0046,0122,writer,"tt0056937,tt0272979,tt0237666,tt0472473"


In [5]:
actors.shape # table brute

(11368358, 6)

## Table Movies

In [6]:
# appel de la table films (j'ai pris title basics qui contient l'année du film)

movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', low_memory=False)
movies.sort_values('startYear', inplace=True)

In [7]:
#reset index base movies
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [8]:
# check si l'ordre est bien modifié et l'index modifié
movies.head() 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt3155794,short,Passage de Venus,Passage de Venus,0,1874,\N,1,"Documentary,Short"
1,tt14495706,short,La Rosace Magique,La Rosace Magique,0,1877,\N,1,"Animation,Short"
2,tt12592084,short,Le singe musicien,Le singe musicien,0,1878,\N,1,"Animation,Short"
3,tt2221420,short,Sallie Gardner at a Gallop,Sallie Gardner at a Gallop,0,1878,\N,1,"Documentary,Short"
4,tt15320514,short,Skeleton of Horse,Skeleton of Horse,0,1881,\N,1,"Animation,Short"


In [9]:
movies.shape # table brute

(8479240, 9)

## Table Title

In [10]:
# appel de la table title
title = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")

In [11]:
# filtre par category actor and actress
title = title.loc[(title['category']=='actor')|(title['category']=='actress')]

In [12]:
#reset index base title
title.reset_index(inplace=True)

#drop colonne index en double 
title.drop('index',axis = 1, inplace=True)

In [13]:
# remplacer les \n par des nan
title.replace('\\N', np.nan, inplace=True)

In [14]:
# check si l'index est bien modifié
title.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000005,1,nm0443482,actor,NaN,"[""Blacksmith""]"
1,tt0000005,2,nm0653042,actor,NaN,"[""Assistant""]"
2,tt0000007,1,nm0179163,actor,NaN,NaN
3,tt0000007,2,nm0183947,actor,NaN,NaN
4,tt0000008,1,nm0653028,actor,NaN,"[""Sneezing Man""]"


## 2-Nettoyage de la base Movies :
###          - Remplacer les \N par des nan
###          - Conservation uniquement des films titleType = movie
###          - Retrait des films isadult
###          - Suppresion des colonnes qui ne me sont plus utiles
###          - suppression des lignes aux données startYear et runtimeMinutes avec des nan
###          - change str en int sur les colonnes startYear et runtimeMinutes et tri sur durée + reset index
###          - Tri des valeurs par nconst + set index

In [15]:
# Remplacer les \N par des nan

movies.replace('\\N', np.nan, inplace=True)

In [16]:
#Conservation uniquement des films titleType = movie

movies = movies[movies['titleType'] == 'movie']

In [17]:
# Retrait des films isadult 

movies = movies[movies['isAdult'] == '0']

In [18]:
# supression des colonnes inutiles
movies = movies.drop(columns=['primaryTitle','titleType', 'endYear', 'isAdult'])

#movies = movies.drop(columns=['primaryTitle','originalTitle','titleType', 'endYear', 'isAdult', 'genres'])

In [19]:
movies.head(2)

,tconst,originalTitle,startYear,runtimeMinutes,genres
682,tt2210499,Birmingham,1896,61,Documentary
1488,tt0229676,Reproduction of the Corbett and Fitzsimmons Fight,1897,NaN,"Documentary,News,Sport"


In [20]:
# suppression des lignes aux données startYear et runtimeMinutes avec des nan

movies.dropna(subset=['startYear', 'runtimeMinutes' ], inplace=True)



In [21]:
# change str en int sur les colonnes startYear et runtimeMinutes
movies = movies.astype({'startYear': int})
movies = movies.astype({'runtimeMinutes': int})

In [22]:
#je ne conserve que les durées entre 58mn et inf à 270mn
movies = movies[(movies['runtimeMinutes']>=58) & (movies['runtimeMinutes']<=270)]

In [23]:
#reset index base movies (again !)
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [24]:
# trie par valeur nconst
movies = movies.sort_values('tconst')

#set_index = nconst
movies = movies.set_index('tconst')

In [25]:
#check
movies.head(2)

,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,
tt0000502,Bohemios,1905,100,NaN
tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"


In [26]:
# table uniquement avec les données conservées
movies.shape 

(326163, 4)

## 3- Nettoyage de la base Actors :
###        - Remplacer les \N par des nan
###        - Supprimer les lignes qui contiennent des nan dans la colonne birthYear  
###        - Retirer les colonnes qui ne me sont pas utiles
###        - Supprimer les acteurs nés avant 1800 + reset index
###        - change str en int sur les colonnes startYear et runtimeMinutes
###        - Tri des valeurs par nconst + set index

In [27]:
# remplacer les \N pas des nan 

actors.replace('\\N', np.nan, inplace=True)

In [28]:
# suppression des lignes qui comportent des Nan dans la colonne birthYear

actors.dropna(subset=['birthYear'], inplace=True)

In [29]:
# Retirer les colonnes qui ne me sont pas utiles
#actors.drop(columns=['deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [30]:
# ne conserve que les rows des acteurs/actrices nés avant 1800
actors = actors[actors['birthYear'] >= '1800']

#reset index base actors (again !)
actors.reset_index(inplace=True)

#drop colonne index en double (again !)
actors.drop('index',axis = 1, inplace=True)

In [31]:
# change str en int sur les colonnes startYear et runtimeMinutes
actors = actors.astype({'birthYear': int})

In [32]:
# Table actors - trie par valeur nconst
actors = actors.sort_values('nconst')

#set_index = nconst
actors = actors.set_index('nconst')

In [33]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536849 entries, nm0000001 to nm9993675
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   primaryName        536849 non-null  object
 1   birthYear          536849 non-null  int64 
 2   deathYear          183622 non-null  object
 3   primaryProfession  470277 non-null  object
 4   knownForTitles     523878 non-null  object
dtypes: int64(1), object(4)
memory usage: 24.6+ MB


In [34]:
# check index
actors.head()

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0117057,tt0038355,tt0037382"
nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0054452,tt0057345"
nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0080455,tt0072562,tt0077975"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0060827,tt0050986,tt0050976"


In [35]:
# explode colonne primaryProfession
#actors[['firstProfession', 'secondProfession', 'thirdProfession']] = actors['primaryProfession'].str.split(',', n=2, expand=True)

#drop colonne primaryProfession
#actors.drop(columns=['primaryProfession'],axis = 1, inplace=True)

In [36]:
# check dataframe actors
#actors.head()

In [37]:
# Je ne garde que les acteurs/actrices de mes trois colonnes : firstProfession, secondProfession, thirdProfession

#actors = actors[(actors['firstProfession'].isin(['actress','actor'])) | ( actors['secondProfession'].isin(['actress','actor'])) | ( actors['thirdProfession'].isin(['actress','actor']))]

In [38]:
#drop colums firstProfession, secondProfession, thirdProfession qui ne me sont plus utiles
#actors.drop(columns=['firstProfession','secondProfession', 'thirdProfession'],axis = 1, inplace=True)

## 3- Nettoyage de la base Title :
###        - Remplacer les \N par des nan
###        - dropna sur les cellules tconst non renseignées
###        - supprimer les colonnes inutiles
###        - Tri des valeurs par nconst + set index

In [39]:
# Remplacer les \N par des nan

title.replace('\\N', np.nan, inplace=True)

In [40]:
#dropna sur les cellules startyear non renseignées

title.dropna(subset=['tconst'],inplace=True)

In [41]:
title.head(2)

,tconst,ordering,nconst,category,job,characters
0,tt0000005,1,nm0443482,actor,NaN,"[""Blacksmith""]"
1,tt0000005,2,nm0653042,actor,NaN,"[""Assistant""]"


In [42]:
# suppression des colonnes inutiles

title.drop(columns=['ordering','job', 'characters'], inplace=True)

In [43]:
# trie par valeur nconst
title = title.sort_values('nconst')

#set_index = nconst
title = title.set_index('nconst')

In [44]:
title.head()

,tconst,category
nconst,,
nm0000001,tt0055307,actor
nm0000001,tt0031983,actor
nm0000001,tt0029971,actor
nm0000001,tt0507706,actor
nm0000001,tt0038370,actor


## 3- Joindre les tables et retrier :

In [45]:
# Table actors
actors.head(2)


,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0117057,tt0038355,tt0037382"


In [46]:
# table title:
title.head(2)

,tconst,category
nconst,,
nm0000001,tt0055307,actor
nm0000001,tt0031983,actor


In [47]:
# merge des tables actors et title 
df_actors_title = actors.merge(title , how='inner', on='nconst')

In [48]:
#check merge actors et title 
df_actors_title.head()

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,category
nconst,,,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0055307,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0031983,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0029971,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0507706,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0038370,actor


In [49]:
#reset index de la table merge

df_actors_title = df_actors_title.reset_index()

In [50]:
#check
df_actors_title.head(2)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,category
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0055307,actor
1,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",tt0031983,actor


In [51]:
#set index de la table merge avec la colonne tconst (pour mon prochain merge)
df_actors_title = df_actors_title.set_index('tconst')

In [52]:
#check
df_actors_title.head(2)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category
tconst,,,,,,,
tt0055307,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor
tt0031983,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor


In [53]:
# Table  movies
movies.head(2)

,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,
tt0000502,Bohemios,1905,100,NaN
tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"


In [54]:
# merge des tables df_actors_title  et movies

actors_title_movies = df_actors_title.merge(movies, how='inner', on='tconst' )


In [55]:
# check merge actors_title  et movies
actors_title_movies.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,,,,,,,,
tt0055307,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor,The Pleasure of His Company,1961,115,Comedy
tt0055307,nm0001666,Debbie Reynolds,1932,2016,"actress,soundtrack,writer","tt0045152,tt0119360,tt0051051,tt0058708",actress,The Pleasure of His Company,1961,115,Comedy
tt0055307,nm0002147,Tab Hunter,1931,2018,"actor,producer,soundtrack","tt0049036,tt0051516,tt0084021,tt0082926",actor,The Pleasure of His Company,1961,115,Comedy
tt0055307,nm0658339,Lilli Palmer,1914,1986,"actress,soundtrack,writer","tt0051964,tt0044631,tt0056129,tt0048948",actress,The Pleasure of His Company,1961,115,Comedy
tt0031983,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor,The Story of Vernon and Irene Castle,1939,93,"Biography,Drama,Musical"


In [56]:
# calcul age acteur au moment du film + creation colonne 'age_actor_actress' pour le résultat

actors_title_movies['age_actor_actress'] = actors_title_movies['startYear'] - actors_title_movies['birthYear']


In [57]:
actors_title_movies.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category,originalTitle,startYear,runtimeMinutes,genres,age_actor_actress
tconst,,,,,,,,,,,,
tt0055307,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor,The Pleasure of His Company,1961,115,Comedy,62
tt0055307,nm0001666,Debbie Reynolds,1932,2016,"actress,soundtrack,writer","tt0045152,tt0119360,tt0051051,tt0058708",actress,The Pleasure of His Company,1961,115,Comedy,29
tt0055307,nm0002147,Tab Hunter,1931,2018,"actor,producer,soundtrack","tt0049036,tt0051516,tt0084021,tt0082926",actor,The Pleasure of His Company,1961,115,Comedy,30
tt0055307,nm0658339,Lilli Palmer,1914,1986,"actress,soundtrack,writer","tt0051964,tt0044631,tt0056129,tt0048948",actress,The Pleasure of His Company,1961,115,Comedy,47
tt0031983,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor,The Story of Vernon and Irene Castle,1939,93,"Biography,Drama,Musical",40


## nettoyage df final : actors_title_movies

In [58]:
#reset index de la table pour que la Pk soit unique
actors_title_movies = actors_title_movies.reset_index()

In [59]:
#check
actors_title_movies.head()

,tconst,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category,originalTitle,startYear,runtimeMinutes,genres,age_actor_actress
0,tt0055307,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor,The Pleasure of His Company,1961,115,Comedy,62
1,tt0055307,nm0001666,Debbie Reynolds,1932,2016,"actress,soundtrack,writer","tt0045152,tt0119360,tt0051051,tt0058708",actress,The Pleasure of His Company,1961,115,Comedy,29
2,tt0055307,nm0002147,Tab Hunter,1931,2018,"actor,producer,soundtrack","tt0049036,tt0051516,tt0084021,tt0082926",actor,The Pleasure of His Company,1961,115,Comedy,30
3,tt0055307,nm0658339,Lilli Palmer,1914,1986,"actress,soundtrack,writer","tt0051964,tt0044631,tt0056129,tt0048948",actress,The Pleasure of His Company,1961,115,Comedy,47
4,tt0031983,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308",actor,The Story of Vernon and Irene Castle,1939,93,"Biography,Drama,Musical",40


In [60]:
#drop colonnes qui m'ont servi uniquement à vérifier mes merges
actors_title_movies.drop(columns=['tconst', 'nconst', 'primaryProfession','knownForTitles','runtimeMinutes', 'genres'], inplace=True)

In [61]:
#check
actors_title_movies.head()

,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
0,Fred Astaire,1899,1987,actor,The Pleasure of His Company,1961,62
1,Debbie Reynolds,1932,2016,actress,The Pleasure of His Company,1961,29
2,Tab Hunter,1931,2018,actor,The Pleasure of His Company,1961,30
3,Lilli Palmer,1914,1986,actress,The Pleasure of His Company,1961,47
4,Fred Astaire,1899,1987,actor,The Story of Vernon and Irene Castle,1939,40


In [62]:
# je tri ici car cela permet de trier toutes les tables d'uns eul et éviter de faire 3 fois la même opé

actors_title_movies = actors_title_movies[(actors_title_movies['startYear'] >= 1918) & (actors_title_movies['startYear'] <= 2021)]

In [63]:
#check
actors_title_movies.shape

(658640, 7)

In [64]:
#film pour checker ensuite la moyenne

check_film = actors_title_movies[actors_title_movies['originalTitle'] == "Easter Parade"]


In [65]:
check_film

,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
122,Fred Astaire,1899,1987,actor,Easter Parade,1948,49
123,Judy Garland,1922,1969,actress,Easter Parade,1948,26
124,Peter Lawford,1923,1984,actor,Easter Parade,1948,25
125,Ann Miller,1923,2004,actress,Easter Parade,1948,25


### groupeby originalTitle sur moyenne des ages des acteurs (tous genres confondus)
### même chose pour les acteurs et actrices

In [66]:
# calcul moyenne age groupby('originalTitle')
age_mean = round(actors_title_movies.groupby('originalTitle').mean('age_actor_actress'))

# check df age_mean
age_mean.shape


(200475, 3)

In [67]:
# calcul moyenne age groupby(by=["originalTitle" & "startYear"])
age_mean2 = round(actors_title_movies.groupby(by=["originalTitle"]).mean('age_actor_actress'))

age_mean2 = age_mean2.groupby('startYear')

# check df age_mean
age_mean2.head()

,birthYear,startYear,age_actor_actress
originalTitle,,,
#1 Serial Killer,1982.0,2013.0,31.0
"#73, Shaanthi Nivaasa",1962.0,2007.0,45.0
#BerhentiDiKamu,1952.0,2021.0,69.0
"#Bitch, les filles et la violence",1965.0,2015.0,50.0
#BlackSkin,1972.0,2021.0,49.0
...,...,...,...
A Social Celebrity,1889.0,1926.0,37.0
A Splendid Hazard,1882.0,1920.0,38.0
A Thousand to One,1879.0,1920.0,41.0


In [81]:
type(age_mean2)

pandas.core.frame.DataFrame

In [68]:
#reset index
age_mean2 = age_mean.reset_index()
age_mean2.head()

,originalTitle,birthYear,startYear,age_actor_actress
0,#1 Serial Killer,1982.0,2013.0,31.0
1,"#73, Shaanthi Nivaasa",1962.0,2007.0,45.0
2,#BerhentiDiKamu,1952.0,2021.0,69.0
3,"#Bitch, les filles et la violence",1965.0,2015.0,50.0
4,#BlackSkin,1972.0,2021.0,49.0


In [69]:
age_mean2.drop(columns=['birthYear'], inplace=True)

In [70]:
#check df age_mean2
age_mean2.head()

,originalTitle,startYear,age_actor_actress
0,#1 Serial Killer,2013.0,31.0
1,"#73, Shaanthi Nivaasa",2007.0,45.0
2,#BerhentiDiKamu,2021.0,69.0
3,"#Bitch, les filles et la violence",2015.0,50.0
4,#BlackSkin,2021.0,49.0


In [71]:
#check shape age_mean2
age_mean2.shape

(200475, 3)

In [72]:
# selection du film : Easter Parade	
age_mean_film = age_mean2[age_mean2['originalTitle'] == "Easter Parade"]
age_mean_film

,originalTitle,startYear,age_actor_actress
45948,Easter Parade,1948.0,31.0


In [73]:
#selection de tous les films de 2015, age moy >= 50

age_actor_film_2015 = age_mean2[(age_mean2['startYear'] == 2015) & (age_mean2['age_actor_actress'] >= 50)]
age_actor_film_2015

,originalTitle,startYear,age_actor_actress
3,"#Bitch, les filles et la violence",2015.0,50.0
35,#artoffline,2015.0,62.0
374,1.7 Alpha,2015.0,56.0
387,10 Days in a Madhouse,2015.0,56.0
902,2 Premi Premache,2015.0,56.0
...,...,...,...
198815,Zombieworld,2015.0,50.0
198838,Zonda: folclore argentino,2015.0,61.0
198865,Zoo School,2015.0,55.0
200127,Étoile montante,2015.0,55.0


## Table actor et table actress

In [74]:
actors = actors_title_movies[actors_title_movies['category'] == "actor"]
actress = actors_title_movies[actors_title_movies['category'] == "actress"]


In [75]:
actors.head()

,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
0,Fred Astaire,1899,1987,actor,The Pleasure of His Company,1961,62
2,Tab Hunter,1931,2018,actor,The Pleasure of His Company,1961,30
4,Fred Astaire,1899,1987,actor,The Story of Vernon and Irene Castle,1939,40
5,Walter Brennan,1894,1974,actor,The Story of Vernon and Irene Castle,1939,45
8,Fred Astaire,1899,1987,actor,Carefree,1938,39


In [76]:
actress.head()

,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
1,Debbie Reynolds,1932,2016,actress,The Pleasure of His Company,1961,29
3,Lilli Palmer,1914,1986,actress,The Pleasure of His Company,1961,47
6,Ginger Rogers,1911,1995,actress,The Story of Vernon and Irene Castle,1939,28
7,Edna May Oliver,1883,1942,actress,The Story of Vernon and Irene Castle,1939,56
10,Ginger Rogers,1911,1995,actress,Carefree,1938,27


In [89]:
# calcul moyenne age groupby(by=["originalTitle" & "startYear"])
age_mean_actors = round(actors.groupby(by=["originalTitle"]).mean('age_actor_actress'))
age_mean_actors = age_mean_actors.groupby('startYear').mean('age_actor_actress')

age_mean_actors.reset_index(inplace=True)
age_mean_actors.drop('birthYear', axis = 1, inplace=True)


In [90]:
age_mean_actors.head()

,startYear,age_actor_actress
0,1918.0,39.132075
1,1919.0,39.948187
2,1920.0,39.569672
3,1921.0,39.777778
4,1922.0,39.625000
